<a href="https://colab.research.google.com/github/guptaaryanr/btc-bots/blob/main/bitcoin_trading_botv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install websocket-client

In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

In [ ]:
import websocket
import talib
import numpy as np
import json

In [ ]:
symbol = 'btcusdt'
interval = '1m'
socket = f'wss://stream.binance.com:9443/ws/{symbol}@kline_{interval}'
core_to_trade = True
amount = 1000
core_amount = amount * 0.8
trade_amount = amount * 0.2
core_quantity = 0
money_end = amount
portfolio = 0
investment = []
closes = []
highs = []
lows = []
opens = []
real_time_portfolio_value = []
engulfing = []
last_eng, amt, port_value, trade_amt, rt_port = 0, 0, 0, 0, 0

In [ ]:
def buy(allocated_money, price):
  global portfolio, money_end, investment
  quantity = allocated_money / price
  money_end -= (quantity * price)
  portfolio += quantity
  if investment == []:
      investment.append(allocated_money)
  else:
      investment.append(allocated_money)
      investment[-1] += investment[-2]
      
def sell(allocated_money, price):
  global portfolio, money_end, investment
  quantity = allocated_money / price
  money_end += (quantity * price)
  portfolio -= quantity
  investment.append(-allocated_money)
  investment[-1] += investment[-2]

In [ ]:
def on_message(ws, message):
  global closes, highs, lows, core_to_trade, core_quantity, money_end
  global portfolio, investment, real_time_portfolio_value
  json_message = json.loads(message)
  cs = json_message['k']
  candle_closed = cs['x']
  close = cs['c']
  high = cs['h']
  low = cs['l']
  open = cs['o']

  if candle_closed:
    closes.append(float(close))
    highs.append(float(high))
    lows.append(float(low))
    opens.append(float(open))
    print(f'Closes: {closes}')

    if core_to_trade:
      buy(core_amount, price = closes[-1])
      print(f'Core Investment: Bought ${core_amount} BTC')
      core_quantity += (core_amount / closes[-1])
      core_to_trade = False

    engulfing = talib.CDLENGULFING(np.array(opens), np.array(highs), np.array(lows), np.array(closes))
    last_eng = engulfing[-1]
    amt = last_eng * trade_amount / 100
    port_value = portfolio * closes[-1] - core_quantity * closes[-1]
    trade_amt = amt - port_value

    rt_port = portfolio * closes[-1] + money_end
    real_time_portfolio_value.append(float(rt_port))
    print(f'Last Engulfing Value is {last_eng} and recommended exposure is ${amt}')
    print(f'Real-Time Portfolio Value: ${rt_port}')

    if trade_amt > 0:
      buy(trade_amt, closes[-1])
      print(f'Bought ${trade_amt} BTC')
    elif trade_amt < 0:
      sell(-trade_amt, closes[-1])
      print(f'Sold ${trade_amt} BTC')

    print('\n')

ws = websocket.WebSocketApp(socket, on_message = on_message)

In [ ]:
ws.run_forever()

In [ ]:
investment

In [ ]:
portfolio_value = portfolio * closes[-1]
if portfolio_value > 0:
    sell(portfolio_value, closes[-1])
else:
    buy(-portfolio_value, closes[-1])
money_end += investment[-1]
print('All trades settled')

In [ ]:
portfolio

In [ ]:
money_end

In [ ]:
btc_return = closes[-1] / closes[0] - 1
bot_return = real_time_portfolio_value[-1] / amount - 1
alpha = bot_return - btc_return
btc_risk = np.std(np.log(np.array(closes[1:]) / np.array(closes[:-1])))
bot_risk = np.std(np.log(np.array(real_time_portfolio_value[1:]) / np.array(real_time_portfolio_value[:-1])))
btc_sharpe = round(btc_return / bot_risk, 3)
bot_sharpe = round(bot_return / bot_risk, 3)
print(btc_return, ',', btc_risk, ',', btc_sharpe)
print(bot_return, ',', bot_risk, ',', bot_sharpe)
print(alpha)